!@pip install modelscope


!pip install modelscope

In [10]:

import torch;



# 示例数据
new_data = torch.randn(1, 3, 224, 224) # 示例数据



print(f"new_data.shape:{new_data.shape}" );

new_data




new_data.shape:torch.Size([1, 3, 224, 224])


tensor([[[[-1.3281,  0.1886,  0.0079,  ...,  0.4142,  0.3857, -0.4796],
          [-1.8078,  0.4325, -0.2976,  ..., -0.2563, -0.1678,  0.7117],
          [-0.4261,  0.3471,  0.7903,  ..., -1.0470,  0.8813,  1.1001],
          ...,
          [ 0.0899,  0.2343,  0.1978,  ...,  0.1238, -1.1190,  0.7299],
          [ 0.1859, -2.0315, -0.8164,  ...,  1.6072, -0.6566, -0.6520],
          [-1.0012, -1.1342,  1.4717,  ...,  2.4817,  1.6954, -0.2797]],

         [[ 0.1025, -0.9869,  1.4794,  ..., -0.3101,  0.9283, -0.5851],
          [ 0.6504,  0.3255, -0.2406,  ...,  1.3563, -0.3143,  0.5066],
          [ 0.9018, -0.9796,  2.0032,  ..., -0.9022,  0.5534, -0.0470],
          ...,
          [-1.2150, -1.2901, -0.5150,  ..., -0.0085, -0.0691, -1.7975],
          [ 0.0955, -0.7213, -1.8479,  ..., -0.1106, -2.0152,  1.2165],
          [ 1.0462, -0.1048,  2.1170,  ..., -0.0128,  0.7187,  0.9816]],

         [[-1.5909,  0.3048,  0.3829,  ...,  0.1964, -0.3818,  0.0909],
          [-0.2021, -1.2225, -

In [ ]:
from timm.data import create_transform;
# 创建数据转换器


transform = create_transform(input_size=new_data, transform=lambda x: x.float());


transform

TypeError: create_transform() got an unexpected keyword argument 'transform'

In [2]:
import torch 
from PIL import Image

import cn_clip.clip as clip
from cn_clip.clip import load_from_name, available_models
print("Available models:", available_models())  
# Available models: ['ViT-B-16', 'ViT-L-14', 'ViT-L-14-336', 'ViT-H-14', 'RN50']

device = "cuda" if torch.cuda.is_available() else "cpu"
# 如本地模型不存在，自动从ModelScope下载模型，需要提前安装`modelscope`包
model, preprocess = load_from_name("ViT-B-16", device=device, download_root='./', use_modelscope=True)
model.eval()
image = preprocess(Image.open("examples/pokemon.jpeg")).unsqueeze(0).to(device)
text = clip.tokenize(["杰尼龟", "妙蛙种子", "小火龙", "皮卡丘"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    # 对特征进行归一化，请使用归一化后的图文特征用于下游任务
    image_features /= image_features.norm(dim=-1, keepdim=True) 
    text_features /= text_features.norm(dim=-1, keepdim=True)    

    logits_per_image, logits_per_text = model.get_similarity(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # [[1.268734e-03 5.436878e-02 6.795761e-04 9.436829e-01]]

Available models: ['ViT-B-16', 'ViT-L-14', 'ViT-L-14-336', 'ViT-H-14', 'RN50']


RuntimeError: Ckpt download requires `modelscope`. Please install `modelscope` or download the ckpt manually and provide the local path so that we can continue.